In [55]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pymongo


In [57]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

In [58]:
db = client.test_db
# Alternatively db= client["test-db"]

test_col = db.test_col
#Alternatively test_col = db["test-col"]


In [59]:
import datetime

title = "This is my first mongo insert"
content = "This has the content of the blog"
img_id = ["id for first img", "id for the second img","Id for the third img"]
writting_date = datetime.datetime.now()
author = "arc"


post = {
        "title":title,
        "content":content,
        "author":author,
        "img_id":img_id,
        "writting_date":writting_date,
        "timestamp_of_insertion":datetime.datetime.now()
}

In [60]:
posts = db.posts
post_id = posts.insert_one(post).inserted_id

print(post_id)



68037b1dcbc53b9c6e7f72ff


In [61]:
print(db.list_collection_names())

['posts']


In [62]:
pip install pprint

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint
Note: you may need to restart the kernel to use updated packages.


In [63]:
import pprint as prettyprint # type: ignore

# prettyprint.pprint(posts.find_one(post_id))

## Short note on BSON
---
- BSON stands for binary JSON
- Used for storing RPC(remote procedure call) and documents
- some examples of types not avalable in JSON but in BSON are 
    - ObjectID
    - Data
    - Binary data
- Porvides faster traversal of encoded data

### Why?
- 12 bytes ObjectId by mongo containing
    - A 4-byte timestamp, representing the ObjectId's creation, measured in seconds since the Unix epoch.

    - A 5-byte random value generated once per process. This random value is unique to the machine and process.

    - A 3-byte incrementing counter, initialized to a random value

- this gurantees uniqueness


In [64]:
from bson.objectid import ObjectId

#note how we have to first convert to the BSON ObjectId tp query
prettyprint.pprint(posts.find_one({'_id':ObjectId('680375bacbc53b9c6e7f72fa')}))

{'_id': ObjectId('680375bacbc53b9c6e7f72fa'),
 'author': 'arc',
 'content': 'This has the content of the blog',
 'img_id': ['id for first img',
            'id for the second img',
            'Id for the third img'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000),
 'title': 'This is my first mongo insert',
 'writting_date': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000)}


In [65]:
#inserting a list of JSON
list_of_posts = [
    {
        "title":"tis the second post",
        "content":"tis the content of second post",
        "img_id":["12331231"],
        "author":"arc",
        "writting_date":datetime.datetime.now(),
        "timestamp_of_insertion": datetime.datetime.now()
    },
    {
        "title":"this is the third post",
        "content":"I am exhausted by typing all this dummy text",
        "author": "blah",
        "img_id":["123123132"],
        "writting_date":datetime.datetime.now(),
        "timestamp_of_insertion":datetime.datetime.now()
    }
]

lst_of_id = posts.insert_many(list_of_posts)

In [66]:
#to get if the insertion worked
print(lst_of_id.acknowledged)

#to get the ids of inserted posts
print(lst_of_id.inserted_ids)

True
[ObjectId('68037b1ecbc53b9c6e7f7300'), ObjectId('68037b1ecbc53b9c6e7f7301')]


In [67]:
# Now to find posts

posts_cursor = posts.find()

for post in posts_cursor:
    prettyprint.pprint(post)



{'_id': ObjectId('680375bacbc53b9c6e7f72fa'),
 'author': 'arc',
 'content': 'This has the content of the blog',
 'img_id': ['id for first img',
            'id for the second img',
            'Id for the third img'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000),
 'title': 'This is my first mongo insert',
 'writting_date': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000)}
{'_id': ObjectId('680375edcbc53b9c6e7f72fb'),
 'author': 'arc',
 'content': 'tis the content of second post',
 'img_id': ['12331231'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 37, 41, 296000),
 'title': 'tis the second post',
 'writting_date': datetime.datetime(2025, 4, 19, 15, 37, 41, 296000)}
{'_id': ObjectId('680375edcbc53b9c6e7f72fc'),
 'author': 'blah',
 'content': 'I am exhausted by typing all this dummy text',
 'img_id': ['123123132'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 37, 41, 296000),
 'title': 'this is the third post',
 'writt

In [68]:
#to filter people we do
arc_cursor_posts = posts.find({"author":"arc"})

for post in arc_cursor_posts:
    prettyprint.pprint(post)
    print("\n\n\n-------------------------------------------------\n\n\n")

arc_posts_count = posts.count_documents({"author":"arc"})
print(f"No of posts by Arc are : {arc_posts_count}")

{'_id': ObjectId('680375bacbc53b9c6e7f72fa'),
 'author': 'arc',
 'content': 'This has the content of the blog',
 'img_id': ['id for first img',
            'id for the second img',
            'Id for the third img'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000),
 'title': 'This is my first mongo insert',
 'writting_date': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000)}



-------------------------------------------------



{'_id': ObjectId('680375edcbc53b9c6e7f72fb'),
 'author': 'arc',
 'content': 'tis the content of second post',
 'img_id': ['12331231'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 37, 41, 296000),
 'title': 'tis the second post',
 'writting_date': datetime.datetime(2025, 4, 19, 15, 37, 41, 296000)}



-------------------------------------------------



{'_id': ObjectId('68037b1dcbc53b9c6e7f72ff'),
 'author': 'arc',
 'content': 'This has the content of the blog',
 'img_id': ['id for first img',
            'id f

In [73]:
# range based learch and filter
import datetime
date_before = datetime.datetime(2025, 4, 19,17,12)
date_filter_cursor = posts.find({"writting_date": {"$lt":date_before}}).sort("author")

for post in date_filter_cursor:
    prettyprint.pprint(post)
    print("\n\n\n--------------------------------------------\n\n\n")

{'_id': ObjectId('680375bacbc53b9c6e7f72fa'),
 'author': 'arc',
 'content': 'This has the content of the blog',
 'img_id': ['id for first img',
            'id for the second img',
            'Id for the third img'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000),
 'title': 'This is my first mongo insert',
 'writting_date': datetime.datetime(2025, 4, 19, 15, 36, 44, 961000)}



--------------------------------------------



{'_id': ObjectId('680375edcbc53b9c6e7f72fb'),
 'author': 'arc',
 'content': 'tis the content of second post',
 'img_id': ['12331231'],
 'timestamp_of_insertion': datetime.datetime(2025, 4, 19, 15, 37, 41, 296000),
 'title': 'tis the second post',
 'writting_date': datetime.datetime(2025, 4, 19, 15, 37, 41, 296000)}



--------------------------------------------



{'_id': ObjectId('68037b1dcbc53b9c6e7f72ff'),
 'author': 'arc',
 'content': 'This has the content of the blog',
 'img_id': ['id for first img',
            'id for the sec

In [78]:
#creating indexes for faster query on the particular query

result_index = posts.create_index([("writting_date", pymongo.DESCENDING)])


In [80]:
print(sorted(list(posts.index_information())))

['_id_', 'writting_date_-1']
